# DYNAMIC PROGRAMMING

In [1]:
import numpy as np
import Dynamic_Programming

The different functions used to do this are available in **Dynamic_Programming.py**.
- *OptimalBellmanValue* where we compute $V \mapsto \mathcal{T}V$
- *OptimalBellmanPolicy* where we compute the optimal policy $\pi(x) \in argmax_{a \in A}[r(x,a)+\gamma \sum_y p(y~|~x,a)V^{\pi}(x)]]$
- *PolicyEvaluation* where we compute the Value from any policy $\pi$
- *ValueIteratin* and *PolicyIteratin*
- *PlotConvergenceValue* and *PlotConvergencePolicy* where we plot the convergence of $\|v^{k+1} - v^k\|_{\infty}$ using two different methods.

## Q1: Implement the discrete MDP model 

In [3]:
R = np.array([[-0.4,0],[2,0],[-1,-0.5]])
P = np.array([
            [[0.45,0,0.55],[0,0,1],[0.6,0,0.4]]
            ,
            [[0,0,1],[0.5,0.4,0.1],[0,0.9,0.1]]
            ])
Actions = [0,1]
States = [0,1,2]

Model = Dynamic_Programming.MarkovDecisionProcess(States,Actions,P,R,0.95)

## Q2: Value Iteration

We implement and run value iteration (VI) in order to identify a 0.01-optimal policy. 

## REINFORCEMENT LEARNING